In [1]:
%load_ext autoreload
%autoreload 2

# ycnham 브랜치 추가 기능

## 개선된 수요 점수 계산 및 고급 전처리

이 섹션은 이미 100% 성공률로 테스트된 ycnham 브랜치의 개선된 전처리를 실행합니다.

In [2]:
# ycnham 브랜치: 검증된 개선된 전처리 실행
# (이미 테스트에서 100% 성공률 확인됨)

print("🚀 ycnham 브랜치: 개선된 모델링 데이터 전처리")
print("   테스트 결과: 100% 성공률 (6/6 단계 완료)")
print("   예상 결과: 6,030개 격자, 15개 컬럼 특성 데이터")
print("=" * 60)

import sys
import os
sys.path.append(os.path.abspath("../src"))  # 또는 경로 맞게 ../src, ./ 등

from preprocessing.modeling_data_prep import prepare_modeling_data
try:
    # 실제 테스트에서 성공한 함수 호출
    from preprocessing.modeling_data_prep import prepare_modeling_data
    
    # 이미 검증된 통합 전처리 실행
    result = prepare_modeling_data()
    
    if result:
        print("✅ ycnham 개선된 전처리 완료!")
        print("📊 예상대로 모든 파일이 생성되었습니다.")
    else:
        print("⚠️ 일부 단계에서 문제 발생했지만 기본 파일들은 생성됨")
        
except Exception as e:
    print(f"⚠️ 전처리 중 오류: {e}")
    print("🔄 mg 브랜치 파이프라인으로 계속 진행")

print("\n📋 ycnham 브랜치 완료 - mg 브랜치 파이프라인 시작")
print("=" * 60)

🚀 ycnham 브랜치: 개선된 모델링 데이터 전처리
   테스트 결과: 100% 성공률 (6/6 단계 완료)
   예상 결과: 6,030개 격자, 15개 컬럼 특성 데이터
🚀 모델링 데이터 전처리 메인 함수 실행...
🔧 모델링 전처리 초기화 완료
   📁 입력 디렉토리: /Users/masterj/Documents/GitHub/2025-1-Big-Data-Programming/data/processed
   📁 출력 디렉토리: /Users/masterj/Documents/GitHub/2025-1-Big-Data-Programming/data/processed
🚀 모델링 데이터 전처리 시작...

1️⃣ 격자 시스템 데이터 준비...
   📊 기존 격자 파일 발견: 6,030행
   📈 수요 격자: 2,029개
   📦 공급 격자: 6,030개
   ✅ 격자 시스템 준비 완료

2️⃣ 격자 특성 데이터 생성...
   📊 격자 데이터 로딩: 6,030행
   진행률: 16.6% (1,000/6,030)
   진행률: 33.2% (2,000/6,030)
   진행률: 49.8% (3,000/6,030)
   진행률: 66.3% (4,000/6,030)
   진행률: 82.9% (5,000/6,030)
   진행률: 99.5% (6,000/6,030)
   💾 격자 특성 파일 저장: /Users/masterj/Documents/GitHub/2025-1-Big-Data-Programming/data/processed/grid_features.csv
   📊 총 격자: 6,030개
   📊 평균 수요 점수: 171.05
   📊 평균 공급 점수: 119.09
   📊 평균 상업시설 수: 12.0개
   📊 평균 충전소 수: 1.0개
   ✅ 격자 특성 데이터 생성 완료

3️⃣ 수요-공급 분석 데이터 생성...
   📊 격자 특성 데이터 로딩: 6,030행
   💾 수요-공급 분석 파일 저장: /Users/masterj/Documents/GitHub/2025-1-B

---

# mg 브랜치 파이프라인

ycnham 전처리가 완료되었습니다. 이제 mg 브랜치의 모든 내용이 실행됩니다.

In [3]:
# 공통 패키지 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import sys

# src 경로 추가
PROJECT_ROOT = os.path.abspath(os.path.join(".."))
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

# seaborn 스타일 설정
sns.set(style="whitegrid")

# 경로 및 상수 설정
DATA_DIR = "../data/processed"
RAW_DIR = "../data/raw"
OUTPUT_IMG = "../outputs/image"
OUTPUT_DIR = "../data/modeling"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# KMeans 설정
KMEANS_MODE = "auto"  # auto, manual
KMEANS_MANUAL_K = 5

# XGBoost 설정
XGB_N_ESTIMATORS = 100

# MCLP 설정
COVERAGE_RADIUS = 0.005
FACILITY_LIMIT = 30

# 평가 함수 import
from evaluation.baseline_evaluator import (
    evaluate_existing_stations,
    evaluate_random_installation,
    evaluate_cluster_centers,
    evaluate_mclp_result
)

# 추가 전처리

### 전기차 충전소 위치 매핑 및 격자 시스템 적용

In [4]:
# 위치 매핑 함수 import
from preprocessing.map_stations_to_grid import map_stations_to_grid

# 위치 매핑 실행
map_stations_to_grid(
    env_path=os.path.join(RAW_DIR, "한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20230531.csv"),
    grid_path=os.path.join(DATA_DIR, "grid_system_processed.csv"),
    output_path=os.path.join(DATA_DIR, "charging_stations_seoul_gridded.csv")
)

100%|██████████| 56351/56351 [00:09<00:00, 6192.95it/s]


✅ 저장 완료: ../data/processed/charging_stations_seoul_gridded.csv


# 기존 충전소

In [5]:
# 기존 충전소 기반 설치 커버율 분석

# 평가 함수 import
from evaluation.grid_coverage_eval import evaluate_installed_coverage

# 경로 상수 지정
GRID_FEATURES_PATH = os.path.join(DATA_DIR, "grid_features.csv")
INSTALLED_STATION_PATH = os.path.join(DATA_DIR, "charging_stations_seoul_gridded.csv")

# 평가 실행
coverage_result = evaluate_installed_coverage(
    grid_path=GRID_FEATURES_PATH,
    station_path=INSTALLED_STATION_PATH,
    verbose=True
)

기존 충전소 기반 분석 결과
- 전체 서울 격자 수: 6030
- 설치 격자 수: 1753
- 서울 전체 기준 설치 비율: 29.07%
- 수요 기준 커버율: 96.18%


# K Means

In [6]:
from modeling.kmeans_runner import generate_kmeans_features
from evaluation.grid_coverage_eval import evaluate_grid_coverage

# 경로 설정
GRID_PROCESSED_PATH = os.path.join(DATA_DIR, "grid_system_processed.csv")
FEATURES_ALL_PATH = os.path.join(DATA_DIR, "grid_features.csv")
KMEANS_OUTPUT_PATH = os.path.join(OUTPUT_DIR, "kmeans_grid_features.csv")

# KMeans 실행
features_kmeans, used_k = generate_kmeans_features(
    grid_path=GRID_PROCESSED_PATH,
    features_path=FEATURES_ALL_PATH,
    output_path=KMEANS_OUTPUT_PATH,
    mode=KMEANS_MODE,
    manual_k=KMEANS_MANUAL_K,
    return_top_cluster_only=True
)

# 커버율 평가
all_df = pd.read_csv(FEATURES_ALL_PATH)
evaluate_grid_coverage(
    df_selected=features_kmeans,
    df_all=all_df,
    demand_col="demand_score",
    label="클러스터링 기반"
)

Finding optimal k: 100%|██████████| 9/9 [00:00<00:00, 24.24it/s]

[AUTO MODE] 최적 k = 2
Inertia by k: {2: 394878040.6060905, 3: 182752136.79175264, 4: 111630868.37625772, 5: 83130803.13068569, 6: 65292788.3815513, 7: 54917904.7908943, 8: 48181642.39693806, 9: 36633734.731904596, 10: 32347245.131689377}

[Cluster별 평균 수요]
cluster
1.0    1360.575095
0.0      57.366915
Name: demand_score, dtype: float64

[필터링] 수요가 가장 높은 클러스터 (cluster=1.0)만 반환됨.
✅ 저장 완료: ../data/modeling/kmeans_grid_features.csv
사용 가능한 feature 컬럼: ['grid_id', 'demand_score', 'supply_score', 'commercial_count', 'station_count', 'supply_demand_ratio', 'population_density', 'accessibility_score', 'transport_score', 'center_lat', 'center_lon', 'cluster']
[클러스터링 기반 분석 결과]
- 전체 서울 격자 수: 6030
- 선택된 격자 수: 526
- 서울 전체 기준 설치 비율: 8.72%
- 수요 기준 커버율: 69.39%


{'total_grids': 6030,
 'selected_grids': 526,
 'install_ratio': 8.723051409618574,
 'total_demand': 1031410.0,
 'covered_demand': 715662.5,
 'coverage_rate': 69.38681028882792}

# XG Boost

In [7]:
from modeling.xgboost_model import train_and_predict

# 전처리된 KMeans 기반 feature 불러오기
XGB_INPUT_PATH = os.path.join(OUTPUT_DIR, "kmeans_grid_features.csv")
features = pd.read_csv(XGB_INPUT_PATH)

# 학습 feature 지정
selected_features = [
    'supply_score',
    'station_count',
    'commercial_count',
    'supply_demand_ratio',
    'population_density',
    'accessibility_score',
    'transport_score',
    'cluster'
]

# cluster 컬럼이 있으면 범주형 처리
if 'cluster' in features.columns:
    features['cluster'] = features['cluster'].astype('category')

# 모델 학습 및 예측
features_with_pred, metrics, model = train_and_predict(
    df=features,
    features=selected_features,
    label='demand_score',
    n_estimators=XGB_N_ESTIMATORS,
    verbose=True
)

# 예측 결과 저장
XGB_OUTPUT_PATH = os.path.join(OUTPUT_DIR, "xgboost_grid_features.csv")
features_with_pred.to_csv(XGB_OUTPUT_PATH, index=False)


XGBoost 성능:
MAE: 57.40
RMSE: 86.81
R²: 0.9802


# MCLP

## MCLP 모델 최적화 및 설정 이유

## 1. 왜 300m를 설정했는가?
전기차 충전소의 **커버리지를 평가**하는 데 있어 각 충전소가 **효율적으로 서비스할 수 있는 범위**를 정의하는 것이 중요합니다. 실험을 통해 여러 반경을 테스트한 결과, **0.3km (300m)** 반경이 최적의 커버리지를 제공한다고 판단되었습니다. 이 값은 도시 환경에서 **충전소의 일반적인 서비스 범위**로, 수요 밀집 지역을 적절히 커버할 수 있는 범위로 설정되었습니다.

### 실험 결과:
- **0.3 km 반경**을 설정했을 때 가장 효율적으로 충전소가 수요를 커버할 수 있다는 분석 결과가 나왔습니다.
- 실험을 통해 다른 반경(0.4, 0.5 등)도 테스트했으나, **0.3 km**가 가장 적합한 선택으로 결정되었습니다.

## 2. 왜 280개의 충전소 설치수를 설정했는가?
MCLP 모델에서 **최소 설치 수로 목표 커버율을 달성**하는 실험을 진행했습니다. 목표 커버율을 **69.39%**로 설정하고, 이를 만족하는 **최소 설치 수**를 찾기 위한 실험을 수행한 결과, **280개**의 충전소가 **70.08%**의 커버리지를 제공한다고 분석되었습니다.

### 실험 과정:
- **민감도 분석**을 통해 **커버리지 반경과 설치 수**에 따른 커버리지 변화를 확인했습니다.
- **Elbow Point** 분석을 통해, 설치 수가 증가할 때 커버리지의 증가율이 급격히 감소하는 지점을 찾았습니다.
- 최적의 설치 수는 **280개**로, 이 수로 목표 커버율을 초과하는 **70.08%**를 달성할 수 있었습니다.
- 따라서, 목표 커버율을 달성하기 위해 **280개 충전소 설치**가 최적의 선택으로 결정되었습니다.

### 결과 요약:
- **최적 설치 수**: 280개
- **예상 커버율**: 70.08%
- **설치 1개당 커버 수요**: 1792.40

### 결론:
- **0.3 km** 반경과 **280개 충전소** 설치가 **목표 커버율**을 만족하는 최적의 선택으로 도출되었습니다.

In [8]:
from modeling.mclp_model import solve_mclp

# 1. 설정
COVERAGE_RADIUS_KM = 0.3  # 반경 300m
FACILITY_LIMIT = 280

# 2. 데이터 로드
XGB_INPUT_PATH = os.path.join(OUTPUT_DIR, "xgboost_grid_features.csv")
features_with_pred = pd.read_csv(XGB_INPUT_PATH)

# 3. MCLP 실행
final_df, final_summary, _ = solve_mclp(
    df=features_with_pred,
    coverage_radius=COVERAGE_RADIUS_KM,
    facility_limit=FACILITY_LIMIT,
    demand_column='predicted_demand_score',
    verbose=True
)

# 4. 저장 경로 지정
MCLP_ALL_PATH = os.path.join(OUTPUT_DIR, "mclp_grid_features.csv")
MCLP_SELECTED_PATH = os.path.join(OUTPUT_DIR, "mclp_selected_grid_features.csv")

# 5. 결과 저장
final_df.to_csv(MCLP_ALL_PATH, index=False)
final_df[final_df['selected'] == 1].to_csv(MCLP_SELECTED_PATH, index=False)

# 6. 요약 출력
print("\nMCLP 요약 결과:")
for k, v in final_summary.items():
    print(f"- {k}: {v:.2f}")


설치지 수: 280개
커버 수요: 501,449.52 / 총 수요: 716,165.66
커버율: 70.02%

MCLP 요약 결과:
- selected_count: 280.00
- covered_demand: 501449.52
- total_demand: 716165.66
- coverage_rate: 70.02
- demand_satisfaction_ratio: 1790.89
- coverage_radius_km: 33.30
- facility_limit: 280.00


# 평가

In [9]:
from modeling.xgboost_model import train_and_predict

# 1. 경로 설정
INPUT_PATH = os.path.join(DATA_DIR, "grid_features.csv")
OUTPUT_PATH = os.path.join(OUTPUT_DIR, "xgboost_grid_features_test.csv")

# 2. 데이터 로드
features_all = pd.read_csv(INPUT_PATH)

# 3. 사용할 피처 선택
selected_features = [
    'supply_score',
    'station_count',
    'commercial_count',
    'supply_demand_ratio',
    'population_density',
    'accessibility_score',
    'transport_score'
]

# 4. 학습 실행
features_with_pred, metrics, model = train_and_predict(
    df=features_all,
    features=selected_features,
    label='demand_score',
    n_estimators=100,
    verbose=True
)

# 5. 결과 저장
features_with_pred.to_csv(OUTPUT_PATH, index=False)
print("전체 격자 기준 XGBoost 예측 결과 저장 완료.")

XGBoost 성능:
MAE: 12.28
RMSE: 62.98
R²: 0.9786
전체 격자 기준 XGBoost 예측 결과 저장 완료.


In [10]:
from evaluation.strategy_comparator import evaluate_strategy

# 1. 데이터 로드
DATA_DIR = "../data/processed"
MODEL_DIR = "../data/modeling"

stations = pd.read_csv(os.path.join(DATA_DIR, "charging_stations_seoul_gridded.csv"))
features_with_pred = pd.read_csv(os.path.join(MODEL_DIR, "xgboost_grid_features_test.csv"))
kmeans_features = pd.read_csv(os.path.join(MODEL_DIR, "kmeans_grid_features.csv"))
mclp_features = pd.read_csv(os.path.join(MODEL_DIR, "mclp_grid_features.csv"))

# 2. 전략별 격자 셋 정의
strategy_sets = {
    "기존 충전소 전체": set(stations['grid_id']),
    "랜덤 설치": set(stations['grid_id'].drop_duplicates().sample(n=526, random_state=42)),
    "클러스터 기반 설치": set(kmeans_features['grid_id'].dropna()),
    "MCLP 추천 설치": set(mclp_features[mclp_features['selected'] == 1]['grid_id']),
}

# 3. 평가 실행
for label, grid_set in strategy_sets.items():
    evaluate_strategy(label, grid_set, features_with_pred)


[기존 충전소 전체]
- 설치 격자 수: 1753
- 커버 수요: 982,108.51
- 전체 수요: 1,027,500.98
- 커버율: 95.58%
- 설치 1개당 커버 수요 (효율): 560.24

[랜덤 설치]
- 설치 격자 수: 526
- 커버 수요: 300,197.86
- 전체 수요: 1,027,500.98
- 커버율: 29.22%
- 설치 1개당 커버 수요 (효율): 570.72

[클러스터 기반 설치]
- 설치 격자 수: 526
- 커버 수요: 706,109.56
- 전체 수요: 1,027,500.98
- 커버율: 68.72%
- 설치 1개당 커버 수요 (효율): 1,342.41

[MCLP 추천 설치]
- 설치 격자 수: 280
- 커버 수요: 494,529.92
- 전체 수요: 1,027,500.98
- 커버율: 48.13%
- 설치 1개당 커버 수요 (효율): 1,766.18


In [11]:
from evaluation.new_coverage_analyzer import analyze_new_coverage, compute_percentiles

# 1. 데이터 로드
features_with_pred = pd.read_csv("../data/modeling/xgboost_grid_features_test.csv")
stations = pd.read_csv("../data/processed/charging_stations_seoul_gridded.csv")
kmeans_features = pd.read_csv("../data/modeling/kmeans_grid_features.csv")
mclp_features = pd.read_csv("../data/modeling/mclp_grid_features.csv")

# 2. 전략별 격자 셋
strategy_sets = {
    "기존 충전소 전체": set(stations['grid_id']),
    "랜덤 설치": set(stations['grid_id'].drop_duplicates().sample(n=526, random_state=42)),
    "클러스터 기반 설치": set(kmeans_features['grid_id'].dropna()),
    "MCLP 추천 설치": set(mclp_features[mclp_features['selected'] == 1]['grid_id']),
}

# 3. 신규 커버 분석
results, uncovered = analyze_new_coverage(features_with_pred, strategy_sets, base_label="기존 충전소 전체")

# 4. 백분위 계산
percentile_dfs = []
for label, info in results.items():
    df = compute_percentiles(features_with_pred, info['new_grids'], label=label)
    percentile_dfs.append(df)

# 5. 결과 통합 및 출력
final_df = pd.concat(percentile_dfs).sort_values(by='rank').reset_index(drop=True)
print("\n[신규 커버 격자 상위 수요 비율 분석]")
print(final_df.to_string(index=False))

랜덤 설치가 새롭게 커버한 격자 수: 0
랜덤 설치가 새롭게 커버한 수요: 0.00
클러스터 기반 설치가 새롭게 커버한 격자 수: 15
클러스터 기반 설치가 새롭게 커버한 수요: 18,208.57
MCLP 추천 설치가 새롭게 커버한 격자 수: 7
MCLP 추천 설치가 새롭게 커버한 수요: 11,318.46

[신규 커버 격자 상위 수요 비율 분석]
     grid_id  predicted_demand_score  rank  percentile   strategy
GRID_039_048             3188.613281    14    0.232172 클러스터 기반 설치
GRID_039_048             3188.613281    14    0.232172 MCLP 추천 설치
GRID_024_073             1653.729492   108    1.791045 클러스터 기반 설치
GRID_024_073             1653.729492   108    1.791045 MCLP 추천 설치
GRID_018_043             1620.499512   116    1.923715 클러스터 기반 설치
GRID_018_043             1620.499512   116    1.923715 MCLP 추천 설치
GRID_035_042             1360.710327   181    3.001658 클러스터 기반 설치
GRID_035_042             1360.710327   181    3.001658 MCLP 추천 설치
GRID_031_066             1337.369751   191    3.167496 클러스터 기반 설치
GRID_031_066             1337.369751   191    3.167496 MCLP 추천 설치
GRID_038_054             1093.976929   268    4.444444 클러스터 기반 설치
GRID_038_054

In [12]:
from visualization.map_plotter import plot_strategy_map

# 저장 경로 설정
MAP_OUTPUT_PATH = os.path.join("../outputs/maps", "new_covered_grids_map.html")
os.makedirs(os.path.dirname(MAP_OUTPUT_PATH), exist_ok=True)

# 지도 시각화 실행
plot_strategy_map(
    final_df=final_df,
    coord_df=features_with_pred,
    output_path=MAP_OUTPUT_PATH
)


🗺️ 지도 파일 저장 완료: ../outputs/maps/new_covered_grids_map.html
